In [92]:
import numpy as np
import pandas as pd

In [93]:
df=pd.read_csv('../TextFiles/smsspamcollection.tsv', sep='\t')

In [94]:
df.head()


,label,message,length,punct
0,ham,"Go until jurong point, crazy.. Available only ...",111,9
1,ham,Ok lar... Joking wif u oni...,29,6
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,155,6
3,ham,U dun say so early hor... U c already then say...,49,6
4,ham,"Nah I don't think he goes to usf, he lives aro...",61,2


In [95]:
df.isnull().sum()

label      0
message    0
length     0
punct      0
dtype: int64

In [96]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   label    5572 non-null   object
 1   message  5572 non-null   object
 2   length   5572 non-null   int64 
 3   punct    5572 non-null   int64 
dtypes: int64(2), object(2)
memory usage: 174.2+ KB


In [97]:
df["label"].value_counts()

ham     4825
spam     747
Name: label, dtype: int64

In [98]:
from sklearn.model_selection import train_test_split

In [99]:
df["message"]

0       Go until jurong point, crazy.. Available only ...
1                           Ok lar... Joking wif u oni...
2       Free entry in 2 a wkly comp to win FA Cup fina...
3       U dun say so early hor... U c already then say...
4       Nah I don't think he goes to usf, he lives aro...
                              ...                        
5567    This is the 2nd time we have tried 2 contact u...
5568                 Will ü b going to esplanade fr home?
5569    Pity, * was in mood for that. So...any other s...
5570    The guy did some bitching but I acted like i'd...
5571                           Rofl. Its true to its name
Name: message, Length: 5572, dtype: object

In [100]:
from nltk.stem import WordNetLemmatizer
lemmatizer=WordNetLemmatizer()

In [101]:

import re
corpus=[]
for i in range(len(df)):
    review = re.sub('[^a-zA-Z]', ' ', df["message"][i])
    review = review.lower()
    review = review.split()
    review = [lemmatizer.lemmatize(word) for word in review]
    review = ' '.join(review)
    corpus.append(review)

In [102]:
type(corpus)

list

In [103]:
X=np.array(corpus)

In [104]:
y=df["label"]

In [105]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.33,random_state=42)

In [106]:
from sklearn.feature_extraction.text import TfidfVectorizer,ENGLISH_STOP_WORDS

In [107]:
tf_idf_vect=TfidfVectorizer()

In [108]:
from sklearn.pipeline import Pipeline

In [109]:
slides=[("tfidf",TfidfVectorizer(stop_words=ENGLISH_STOP_WORDS)),("clf",LinearSVC())]

In [110]:
pipe=Pipeline(slides)

In [111]:
pipe.fit(X_train,y_train)

Pipeline(memory=None,
     steps=[('tfidf', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,...ax_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0))])

In [112]:
pred=pipe.predict(X_test)

In [113]:
pd.DataFrame(pred).head(30)


,0
0,ham
1,ham
2,ham
3,ham
4,ham
5,ham
6,ham
7,ham
8,ham
9,ham


In [114]:
pred_list=list(pred)

In [115]:
y_list=list(y_test)

In [116]:
ana=pd.DataFrame(data={"pred":pred_list,"y_test":y_list})

In [117]:
ana.iloc[1335,:]

pred      ham
y_test    ham
Name: 1335, dtype: object

In [118]:
ls=[]
j=0
for i in range(0, len(pred_list)):
    if pred_list[i]!=y_list[j]:
        ls.append([pred_list[i],y_list[j],i])
    j=j+1

In [135]:
len(ls)

22

In [120]:
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score

In [121]:
confusion_matrix(y_test,pred)

array([[1590,    3],
       [  19,  227]], dtype=int64)

In [122]:
print(classification_report(y_test,pred))

              precision    recall  f1-score   support

         ham       0.99      1.00      0.99      1593
        spam       0.99      0.92      0.95       246

   micro avg       0.99      0.99      0.99      1839
   macro avg       0.99      0.96      0.97      1839
weighted avg       0.99      0.99      0.99      1839



In [123]:
accuracy_score(y_test,pred)

0.988036976617727

In [124]:
pipe.predict(["free entri wkli comp win fa cup final tkt st may text fa receiv entri question std txt rate c appli"])

array(['spam'], dtype=object)

In [125]:
pipe.predict(["and also i ve sorta blown him off a couple time recently so id rather not text him out of the blue looking for weed"])

array(['ham'], dtype=object)

In [126]:
X_test[1]

'and also i ve sorta blown him off a couple time recently so id rather not text him out of the blue looking for weed'

In [127]:
y_test[y_test=="spam"]

401     spam
5567    spam
881     spam
1961    spam
2664    spam
        ... 
579     spam
2224    spam
1691    spam
4060    spam
3792    spam
Name: label, Length: 246, dtype: object

In [128]:
y_test

3245    ham
944     ham
1044    ham
2484    ham
812     ham
       ... 
4944    ham
3313    ham
3652    ham
14      ham
4758    ham
Name: label, Length: 1839, dtype: object

In [129]:
X_test[51]

'free for st week no nokia tone ur mob every week just txt nokia to get txting and tell ur mate www getzed co uk pobox w wq norm p tone'

In [130]:
y_test_l=list(y_test)

In [131]:
y_test_l[51]

'spam'

In [132]:
pipe.predict(['free for st week no nokia tone ur mob every week just txt nokia to get txting and tell ur mate www getzed co uk pobox w wq norm p tone'

])

array(['spam'], dtype=object)